<a href="https://colab.research.google.com/github/Sudhanshugupta26/Renamer-A-AI-Modal/blob/main/Renamer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install tensorflow
!pip install pypdf2
!pip install pdf2image
!pip install fpdf


In [13]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Tensor FLow (neurall netowrk)

In [14]:
!pip install tensorflow


DEMO AI

In [15]:
!pip install pytesseract
!apt-get install tesseract-ocr
!pip install pdf2image
!apt-get install -y poppler-utils

!pip install tensorflow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [16]:
!pip install pytesseract pdf2image tensorflow
!apt-get install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [17]:
!pip install --upgrade pip


In [18]:
!pip install tqdm


In [26]:
import os
import shutil
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from PyPDF2 import PdfReader
import pytesseract
from pdf2image import convert_from_path
from PIL import Image, ImageEnhance

# Paths
training_folder = '/content/drive/MyDrive/Renaming_Training_Set'
renamed_output_folder = '/content/drive/MyDrive/Renamed_Files'

# Ensure the output folder exists
os.makedirs(renamed_output_folder, exist_ok=True)

# Function to extract text from PDFs
def extract_text_from_pdf(pdf_path):
    try:
        # Convert PDF to images (only first page for efficiency)
        images = convert_from_path(pdf_path, dpi=150, first_page=1, last_page=1)
        extracted_text = ""

        for img in images:
            # Enhance image quality before OCR (increase contrast)
            img = img.convert("L")  # Convert to grayscale
            enhancer = ImageEnhance.Contrast(img)
            img = enhancer.enhance(2.0)  # Increase contrast for better OCR
            extracted_text += pytesseract.image_to_string(img)

        return extracted_text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return ""

# Function to prepare the dataset for training
def prepare_renaming_dataset(folder_path):
    data, labels = [], []
    for pdf_file in os.listdir(folder_path):
        if pdf_file.endswith('.pdf'):
            pdf_path = os.path.join(folder_path, pdf_file)
            text = extract_text_from_pdf(pdf_path)
            if text.strip():
                data.append(text)
                labels.append(os.path.splitext(pdf_file)[0])  # Use file name as label
    return data, labels

# Prepare the dataset
texts, labels = prepare_renaming_dataset(training_folder)

# Ensure there's data to train
if not texts or not labels:
    raise ValueError("No valid training data found. Check the dataset folder.")

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(texts)

# Encode labels
label_mapping = {label: idx for idx, label in enumerate(set(labels))}
y = np.array([label_mapping[label] for label in labels])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Logistic Regression classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy * 100:.2f}%")

# Function to classify and rename PDF files
def classify_and_rename(pdf_path, clf, vectorizer, output_folder):
    try:
        # Extract text from the PDF
        text = extract_text_from_pdf(pdf_path)
        if not text.strip():
            print(f"No text extracted from {pdf_path}, skipping.")
            return None

        # Convert text to TF-IDF vector
        vector = vectorizer.transform([text])

        # Predict the label
        predicted_label_idx = clf.predict(vector)[0]
        predicted_label = list(label_mapping.keys())[list(label_mapping.values()).index(predicted_label_idx)]

        # Create the new name from the predicted label and extracted text
        new_name = f"{predicted_label}.pdf"
        new_path = os.path.join(output_folder, new_name)

        # Rename and move the file by completely replacing the old name
        shutil.move(pdf_path, new_path)

        return new_path
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
        return None

# Folder for unsorted files to rename
unsorted_rename_folder = '/content/drive/MyDrive/Unsorted_Files_To_Rename'

# Process and rename files
def process_pdf_files():
    for pdf_file in os.listdir(unsorted_rename_folder):
        if pdf_file.endswith('.pdf'):
            pdf_path = os.path.join(unsorted_rename_folder, pdf_file)
            result = classify_and_rename(pdf_path, clf, vectorizer, renamed_output_folder)
            if result:
                print(f"Renamed and moved: {result}")
            else:
                print(f"Failed to process {pdf_file}")

# Start processing files
process_pdf_files()


Model accuracy: 0.00%
Renamed and moved: /content/drive/MyDrive/Renamed_Files/FINANCIAL MANAGEMENT-2023-Sessional 1.pdf
Renamed and moved: /content/drive/MyDrive/Renamed_Files/DATABASE MANAGEMENT SYSTEMS-2023-Semester.pdf
